# Inhomogeneous Erdos Renyi (IER) Random Network Model

## The Inhomogeneous Erdos-Renyi (IER) Random Network Model is parametrized by a matrix of independent-edge probabilities

As we just said, we're going to keep rolling with 